## Parameter Efficient Fine Tuning of Large Language Models

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install transformers trl datasets peft accelerate bitsandbytes sentencepiece

In [8]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    logging,
    pipeline
)
from peft import LoraConfig, PeftModel, get_peft_config
from trl import SFTTrainer
import gc
from datasets import Dataset
import pandas as pd

## Add your data

In [9]:
data = pd.read_csv("/content/drive/MyDrive/SKRIPSIWET/Fine-Tuning-LLMs/data/medquad.csv")

In [10]:
data.sample(3)

question  \
5190       What are the symptoms of TAR syndrome ?   
8716  What is the outlook for Locked-In Syndrome ?   
2426          What is (are) Chlamydia Infections ?   

                                                 answer             source  \
5190  What are the signs and symptoms of TAR syndrom...               GARD   
8716  While in rare cases some patients may regain c...              NINDS   
2426  Chlamydia is a common sexually transmitted dis...  MPlusHealthTopics   

                focus_area  
5190          TAR syndrome  
8716    Locked-In Syndrome  
2426  Chlamydia Infections

In [11]:
data.isnull().sum()

question       0
answer         5
source         0
focus_area    14
dtype: int64

In [12]:
data.shape

(16412, 4)

In [13]:
data.dropna(inplace=True)

In [14]:
data.shape

(16393, 4)

In [15]:
data.drop("source",axis=1,inplace=True)

## Convert into HuggingFace datasets

In [16]:
from datasets import Dataset

In [17]:
data = Dataset.from_pandas(pd.DataFrame(data=data))

In [18]:
data

Dataset({
    features: ['question', 'answer', 'focus_area', '__index_level_0__'],
    num_rows: 16393
})

In [19]:
#hyperparameters
LORA_ALPHA = 32
LORA_DROPOUT = 0.2
LORA_R = 4

LEARNING_RATE = 1e-4
NUM_EPOCHS = 2
BATCH_SIZE = 4
WEIGHT_DECAY = 0.001
MAX_GRAD_NORM = 0.3
gradient_accumulation_steps = 16
STEPS = 1
OPTIM = "adam"
MAX_STEPS = 200
OUTPUT_DIR = "./results"

## Quantization configuration using Bitsandbytes

In [20]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
torch.cuda.get_device_capability()
device_map = "auto"

## Define model and tokenization

In [21]:
model_name = "pankajmathur/orca_mini_3b"

In [22]:
model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map=device_map,
        )
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/21.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [24]:
torch.cuda.empty_cache()

## Define LoRA adapter

In [25]:
peft_config = LoraConfig(
      lora_alpha= LORA_ALPHA,
      lora_dropout= LORA_DROPOUT,
      r= LORA_R,
      bias="none",
      task_type="CAUSAL_LM",
  )

## Setup training parameters

In [26]:
training_args = TrainingArguments(
      output_dir= OUTPUT_DIR,
      per_device_train_batch_size=BATCH_SIZE,
      gradient_accumulation_steps= gradient_accumulation_steps,
      learning_rate= LEARNING_RATE,
      logging_steps= STEPS,
      num_train_epochs= NUM_EPOCHS,
      max_steps= MAX_STEPS,
)

In [27]:
torch.cuda.empty_cache()

In [28]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field= "question",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_args,
)

Map:   0%|          | 0/16393 [00:00<?, ? examples/s]

In [29]:
trainer.train()

TrainOutput(global_step=200, training_loss=1.7887544828653335, metrics={'train_runtime': 2467.7166, 'train_samples_per_second': 5.187, 'train_steps_per_second': 0.081, 'total_flos': 4781878895923200.0, 'train_loss': 1.7887544828653335, 'epoch': 0.78})

In [30]:
logging.set_verbosity(logging.CRITICAL)
torch.cuda.empty_cache()

In [31]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

## Test the model

### Using Inference pipeline

In [32]:
pipe = pipeline(task="text-generation",model=model,tokenizer=tokenizer,max_length=500)

In [33]:
prompt = "Who is at risk for Prostate Cancer?"

In [34]:
template = f"""<s>[INST] <<SYS>>
You are a honest Medical assistant bot.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>

{prompt} [/INST]
"""

In [35]:
result = pipe(template)

In [36]:
response = result[0]['generated_text']
index = response.find("[/INST]")+len("[/INST]")

In [37]:
print(response[index:].strip())

Prostate cancer is a type of cancer that starts in the prostate gland. The risk of developing prostate cancer increases with age. In fact, the lifetime risk of developing prostate cancer is about 1 in 9 men. African American men have a higher risk of developing prostate cancer than other men. Additionally, men with a family history of prostate cancer are also at a higher risk.


## Without using pipeline

In [38]:
from peft import get_peft_model

In [39]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [41]:
inputs = tokenizer(template, return_tensors="pt").to(device)
model = model.to(device)
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST] <<SYS>>
You are a honest Medical assistant bot.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>

Who is at risk for Prostate Cancer? [/INST]

Prostate cancer is a type of cancer that affects the prostate gland, which is located in the male reproductive system. The risk of developing prostate cancer is higher in older men, especially those who have a family history of the disease. Other risk factors for prostate cancer include a history of chronic urinary tract infections, a previous history of benign prostatic hyperplasia (BPH), a positive family history of prostate cancer, and a high level of prostate-specific antigen (PSA) in

In [49]:
from huggingface_hub import notebook_login

notebook_login()

In [55]:
model.push_to_hub("my-org/devfest_bbsr2023_demo")

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-65ccd6b7-2bf8094a617ec7dc25676a83;57893770-60cd-4249-bd0b-fc5ab79f90ce)

Repository Not Found for url: https://huggingface.co/api/models/my-org/devfest_bbsr2023_demo.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [48]:
!nvidia-smi

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
!nvidia-smi -L